<a href="https://colab.research.google.com/github/gagana2023/NL2SQL/blob/main/NL2SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install transformers
!pip install sentencepiece



In [4]:
# download database Chinook
!wget https://github.com/lerocha/chinook-database/raw/master/ChinookDatabase/DataSources/Chinook_Sqlite.sqlite -O chinook.db


--2025-05-20 14:03:51--  https://github.com/lerocha/chinook-database/raw/master/ChinookDatabase/DataSources/Chinook_Sqlite.sqlite
Resolving github.com (github.com)... 140.82.116.3
Connecting to github.com (github.com)|140.82.116.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/lerocha/chinook-database/master/ChinookDatabase/DataSources/Chinook_Sqlite.sqlite [following]
--2025-05-20 14:03:51--  https://raw.githubusercontent.com/lerocha/chinook-database/master/ChinookDatabase/DataSources/Chinook_Sqlite.sqlite
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1067008 (1.0M) [application/octet-stream]
Saving to: ‘chinook.db’

chinook.db          100%[===================>]   1.02M  --.-KB/s    in 0.0

In [5]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "gaussalgo/T5-LM-Large-text2sql-spider"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.35k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/813 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

In [6]:
question = "What is the average, minimum, and maximum age for all French musicians?"
schema = """
   "stadium" "Stadium_ID" int , "Location" text , "Name" text , "Capacity" int , "Highest" int , "Lowest" int , "Average" int , foreign_key:  primary key: "Stadium_ID" [SEP] "singer" "Singer_ID" int , "Name" text , "Country" text , "Song_Name" text , "Song_release_year" text , "Age" int , "Is_male" bool , foreign_key:  primary key: "Singer_ID" [SEP] "concert" "concert_ID" int , "concert_Name" text , "Theme" text , "Year" text , foreign_key: "Stadium_ID" text from "stadium" "Stadium_ID" , primary key: "concert_ID" [SEP] "singer_in_concert"  foreign_key: "concert_ID" int from "concert" "concert_ID" , "Singer_ID" text from "singer" "Singer_ID" , primary key: "concert_ID" "Singer_ID"
"""

input_text = " ".join(["Question: ",question, "Schema:", schema])

model_inputs = tokenizer(input_text, return_tensors="pt")
outputs = model.generate(**model_inputs, max_length=512)

output_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)

print("SQL Query:")
print(output_text)


SQL Query:
["SELECT avg(age), min(age), max(age) FROM singer WHERE country = 'France'"]


In [7]:
#Test cases for evaluation
test_cases = [
    {
        "question": "What is the average, minimum, and maximum age for all French musicians?",
        "expected_sql": "SELECT avg(Age), min(Age), max(Age) FROM singer WHERE Country = 'France';"
    },
    {
        "question": "List the names of all stadiums with a capacity over 50000.",
        "expected_sql": "SELECT Name FROM stadium WHERE Capacity > 50000;"
    },
    {
        "question": "Which concerts were held in the year 2015?",
        "expected_sql": "SELECT concert_Name FROM concert WHERE Year = '2015';"
    },
    {
        "question": "Get the names and ages of all male singers.",
        "expected_sql": "SELECT Name, Age FROM singer WHERE Is_male = 1;"
    },
    {
        "question": "Show all stadiums located in Paris.",
        "expected_sql": "SELECT Name FROM stadium WHERE Location = 'Paris';"
    },
    {
        "question": "What are the themes of concerts held in stadiums with a capacity above 60000?",
        "expected_sql": "SELECT Theme FROM concert JOIN stadium ON concert.Stadium_ID = stadium.Stadium_ID WHERE Capacity > 60000;"
    },
    {
        "question": "List all singers who have performed in a concert.",
        "expected_sql": "SELECT DISTINCT Name FROM singer JOIN singer_in_concert ON singer.Singer_ID = singer_in_concert.Singer_ID;"
    },
    {
        "question": "Get the number of concerts held each year.",
        "expected_sql": "SELECT Year, COUNT(*) FROM concert GROUP BY Year;"
    },
    {
        "question": "Which singers released their song in 2020?",
        "expected_sql": "SELECT Name FROM singer WHERE Song_release_year = '2020';"
    },
    {
        "question": "Find the names of concerts held in stadiums located in New York.",
        "expected_sql": "SELECT concert_Name FROM concert JOIN stadium ON concert.Stadium_ID = stadium.Stadium_ID WHERE Location = 'New York';"
    }
    ]

In [8]:
# Normalize SQL for exact match check
def normalize_sql(sql):
    return sql.strip().lower().replace(";", "").replace("  ", " ")


In [9]:
# Evaluate exact match accuracy
exact_matches = 0
predictions = []
for case in test_cases:
    input_text = "Question: " + case["question"] + " Schema: " + schema
    model_inputs = tokenizer(input_text, return_tensors="pt")
    outputs = model.generate(**model_inputs, max_length=512)
    predicted_sql = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

    predictions.append((case["question"], predicted_sql))

    if normalize_sql(predicted_sql) == normalize_sql(case["expected_sql"]):
        exact_matches += 1

print(f"\n Exact Match Accuracy: {exact_matches}/{len(test_cases)} = {exact_matches / len(test_cases):.2f}\n")


 Exact Match Accuracy: 3/10 = 0.30



In [10]:
# Execution accuracy
correct_exec = 0

def run_sql(query):
    try:
        cursor.execute(query)
        return cursor.fetchall()
    except:
        return None

for i, (question, prediction) in enumerate(predictions):
    expected = test_cases[i]["expected_sql"]
    pred_result = run_sql(prediction)
    expected_result = run_sql(expected)
    if pred_result == expected_result:
        correct_exec += 1

print(f"\n Execution Accuracy: {correct_exec}/{len(test_cases)} = {correct_exec / len(test_cases):.2f}")


 Execution Accuracy: 10/10 = 1.00
